In [ ]:
# Util Functions
import time
import requests
from urllib.parse import urlparse
import sys
import json
import lxml.html
import csv
from wayback import WaybackClient, memento_url_data, WaybackSession
import itertools
from crawlers.utils import make_request, make_link_absolute, page_grab#, #crawl_wayback

REQUEST_DELAY = 0.5

In [ ]:
# Institute for Fox News


def crawl_fox(base_page, article=set(), video=set()):
    """
    From an initial API query page, run through all possible
    API queries-- putting articles and videos on the pages into
    a set.

    Returns:
    Set of articles and videos
    """
    response = page_grab(base_page)
    json_data = json.loads(response[0].text)
    json_data
    for i in json_data:
        url = make_link_absolute(i["url"], "https://www.foxnews.com/politics")
        if url.startswith("https://www.foxnews.com/politics"):
            article.add(url)
        else:
            video.add(url)
    begin = base_page.find("from") + 5
    end = base_page.find("&media")
    articlenumber = int(base_page[begin:end])
    if articlenumber < 9970:
        articlenumber += 30
        articlenumber = str(articlenumber)
        rev_basepage = (
            base_page[0:begin] + articlenumber + base_page[end : (len(base_page) + 1)]
        )
        crawl_fox(rev_basepage, article, video)
    return article, video


article, video = crawl_fox(
    "https://www.foxnews.com/api/article-search?searchBy=categories&values=fox-news%2Fpolitics&size=30&from=15&mediaTags=primary_politics"
)

article, video

In [ ]:
from wayback import WaybackClient, memento_url_data, WaybackSession
import itertools

client = WaybackClient()
results = client.search("https://www.foxnews.com/politics", match_type="exact")
first10 = list(itertools.islice(results, 50000))
post2015 = []
for i in range(len(first10)):
    if first10[i].timestamp.year >= 2015:
        post2015.append(first10[i].view_url)
post2015
# memento_url_data(record.raw_url)

In [ ]:
length = len(post2015)

In [ ]:
url = "https://www.foxnews.com/politics"


def get_urls(url, session=None):
    """
    This function takes a URLs and returns lists of URLs
    for containing each article on that page.

    Parameters:
        * url:  a URL to a page of articles

    Returns:
        A list of article URLs on that page.
    """
    response = page_grab(url, session)
    urls = []
    container = response[0].cssselect("article")

    for j in container:
        atr = j.cssselect("a")
        if atr and len(atr) > 0:
            href = atr[0].get("href")
            if len(href) > 0:
                urls.append(
                    make_link_absolute(href, "https://www.foxnews.com/politics")
                )
    return urls


def crawl_wayback_fox(url):
    client = WaybackClient()
    results = client.search(url, match_type="exact")
    politics_sec_copies = list(itertools.islice(results, 30000))
    post2015 = []
    for i in range(len(politics_sec_copies)):
        if politics_sec_copies[i].timestamp.year >= 2015:
            articles = get_urls(politics_sec_copies[i].view_url, client)
            post2015 += articles
    filtered = list(
        filter(lambda x: "http://www.foxnews.com/politics/20" in x, post2015)
    )

    # converts archive links back to current article links
    orig_artlinks = []
    for a in filtered:
        orig_artlinks.append(memento_url_data(a)[0])
    return orig_artlinks


archived = crawl_wayback_fox(url)

In [ ]:
def crawl_wayback_fox(url, num_pages):
    session = WaybackSession()
    client = WaybackClient(session)
    results = client.search(url, match_type="exact")

    for i in range(num_pages):
        politics_sec_copies = next(results)
        if politics_sec_copies.timestamp.year >= 2015:
            articles = get_urls(politics_sec_copies.view_url, session)
            post2015 += articles
    filtered = list(
        filter(lambda x: "http://www.foxnews.com/politics/20" in x, post2015)
    )

    # converts archive links back to current article links
    orig_artlinks = []
    for a in filtered:
        orig_artlinks.append(memento_url_data(a)[0])
    return orig_artlinks


archived = crawl_wayback_fox(url)

In [ ]:
# save data via json
json_data = json.dumps(list(article))
json_data


# save as csv
def create_csv(set1, set2, filename):
    """
    turns list of articles and videos into a csv with
    these values as respective columns.
    """
    with open(filename, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Articles", "Video"])
        max_length = max(len(set1), len(set2))
        for i in range(max_length):
            row = [
                list(set1)[i] if i < len(set1) else "",
                list(set2)[i] if i < len(set2) else "",
            ]
            writer.writerow(row)


# Calling the function to create the CSV file
create_csv(article, video, "fox_articles.csv")

In [ ]:
client = WaybackClient()
results = client.search(url, match_type="exact")
politics_sec_copies = list(itertools.islice(results, 30000))

In [ ]:
for i in range(len(politics_sec_copies)):
    if politics_sec_copies[i].timestamp.year >= 2015:
        articles = get_urls(politics_sec_copies[i].view_url)
        post2015 += articles
    filtered = list(
        filter(lambda x: "http://www.foxnews.com/politics/20" in x, post2015)
    )

    # converts archive links back to current article links
    orig_artlinks = []
    for a in filtered:
        orig_artlinks.append(memento_url_data(a)[0])
orig_artlinks

In [82]:
import datetime
def get_urls(url, session=None):
    """
    This function takes a URLs and returns lists of URLs
    for containing each article on that page.

    Parameters:
        * url:  a URL to a page of articles

    Returns:
        A list of article URLs on that page.
    """
    response = page_grab(url, session)
    urls = []
    container = response.cssselect("article")
    for j in container:
        atr = j.cssselect("a")
        if atr and len(atr) > 0:
            href = atr[0].get("href")
            if len(href) > 0:
                urls.append(
                    make_link_absolute(href, "https://web.archive.org/")
                )
    return urls

def crawl_wayback(homepage, break_point, scraper_func, startdate):
    '''
    Take a politics homepage, or any source with a list of articles, finds all 
    copies in the archive, and scrapes all of the article links on that page.
    args:
        homepage- the homepage or politics page we are looking for across time
        break_point- the approx. number of copies in the archive
        scraper_func - the individual function built for scraping that page 
        startdate- the date you would like to begin scraping ('YYYYMMDD')
    returns:
        list of articles from startdate to present
    
    '''
    session= WaybackSession()
    client = WaybackClient(session)
    results = client.search(homepage, match_type="exact", from_date=startdate)
    crosstime_urls = list(itertools.islice(results, break_point))
    post_date_articles = set()
    for i in range(len(crosstime_urls)):
        date=datetime.datetime.strptime(startdate, "%Y%m%d")
        if crosstime_urls[i].timestamp.date() >= date.date():
            articles = scraper_func(crosstime_urls[i].view_url, session)
            print(articles)
            # converts archive links back to current article links
            articles = [memento_url_data(item)[0] for item in articles]
            post_date_articles.update(articles) 
    return post_date_articles

crawl_wayback("http://www.foxnews.com/politics/", 30, get_urls, '20221228')

Fetching https://web.archive.org/web/20221228030425/https://www.foxnews.com/politics
['https://web.archive.org/web/20221228030425/https://www.foxnews.com/politics/house-quickly-approves-spending-bill-flees-dc-christmas-break', 'https://web.archive.org/web/20221228030425/https://www.foxnews.com/politics/chip-roy-blows-up-spending-bill-fiery-floor-speech-destroying-united-states', 'https://web.archive.org/web/20221228030425/https://www.foxnews.com/politics/house-dem-dumps-democrat-senate-forcing-rushed-vote-spending-bill-unacceptable', 'https://web.archive.org/web/20221228030425/https://www.foxnews.com/video/6317883953112', 'https://web.archive.org/web/20221228030425/https://www.foxnews.com/video/6317884758112', 'https://web.archive.org/web/20221228030425/https://www.foxnews.com/video/6317884422112', 'https://web.archive.org/web/20221228030425/https://www.foxnews.com/politics/texas-gov-abbott-hits-back-white-house-criticism-busing-migrants-christmas-eve', 'https://web.archive.org/web/202

{'http://foxnews.com/media/connecticut-community-rallies-together-christmas-food-donations-stolen-like-hallmark-movie',
 'http://foxnews.com/media/newt-gingrich-sounds-alarm-opposition-kevin-mccarthy-speaker-bid-gop-danger-meltdown',
 'http://foxnews.com/media/president-biden-karine-jean-pierres-memorable-battles-press-2022',
 'http://foxnews.com/media/sean-duffy-transportation-chaos-mayor-pete-ignoring-problem',
 'http://foxnews.com/opinion/biden-democrats-7-biggest-bloopers-2022',
 'http://foxnews.com/opinion/dickens-christmas-carol-could-help-americans-revitalize-our-spirit',
 'http://foxnews.com/opinion/pete-buttigiegs-political-future-grounded-forever-after-southwest-holiday-travel-disaster',
 'http://foxnews.com/opinion/worst-thing-about-mitch-mcconnell-trillion-omnibus-spending-package',
 'http://foxnews.com/politics/biden-district-republicans-say-mccarthy-speaker-1-ballot-100-ballots-or-more',
 'http://foxnews.com/politics/biden-pardons-woman-convicted-of-murder-5-others-on-dru